In [1]:
import arcpy
import numpy as np
import time

DATA_PATH=r"C:\Users\mike\Documents\OSU\GEOG581\Project\sf_terrain\MyProject1"

arcpy.env.workspace=DATA_PATH
PROJ_PROJ="PROJCS['WGS_1984_UTM_Zone_10N',GEOGCS['GCS_WGS_1984',\
    DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],\
    PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],\
    PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],\
    PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-123.0],\
    PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]" 



In [2]:
arcpy.env.workspace

'C:\\Users\\mike\\Documents\\OSU\\GEOG581\\Project\\sf_terrain\\MyProject1'

In [4]:
arcpy.management.MosaicToNewRaster("ned\\ned19_n37x75_w122x50_ca_sanfrancisco_topobathy_2010\\ned19_n37x75_w122x50_ca_sanfrancisco_topobathy_2010.img;\
                            ned\\ned19_n37x75_w122x75_ca_sanfrancisco_topobathy_2010\\ned19_n37x75_w122x75_ca_sanfrancisco_topobathy_2010.img;\
                            ned\\ned19_n38x00_w122x50_ca_sanfrancisco_topobathy_2010\\ned19_n38x00_w122x50_ca_sanfrancisco_topobathy_2010.img;\
                            ned\\ned19_n38x00_w122x75_ca_sanfrancisco_topobathy_2010\ned19_n38x00_w122x75_ca_sanfrancisco_topobathy_2010.img",
                            r"C:\Users\mike\Documents\OSU\GEOG581\Project\sf_terrain\MyProject1\MyProject1.gdb", 
                            "sf_ned",     
                            PROJ_PROJ, "32_BIT_FLOAT", 1, 1, 
                            "LAST", "FIRST")



NameError: name 'PROJ_PROJ' is not defined

In [6]:
#SF1 Output Lower and Upper:
# The original dataset has a 32.6 meter step south of 4182500 N.
# We will split this raster at that point then correct the lower portion.

arcpy.management.Clip("dem_sf1\\sf1_output.tif", 
                      "542338.2596 4182500 554890.2596 4184862.271", 
                      "dem_sf1\\sf1_output_upper.tif")

arcpy.management.Clip("dem_sf1\\sf1_output.tif", 
                      "542338.2596 4178828.271 554890.2596 4182499", 
                      "dem_sf1\\sf1_output_lower.tif")

<Result 'dem_sf1\\sf1_output_lower.tif'>

In [30]:
arcpy.management.MosaicToNewRaster("dem_sf1\\sf1_output_lower.tif;\
                                    dem_sf2\\sf2_output_max.tif;\
                                    dems_sf2_1\\sf2_1_output_max.tif;\
                                    dem_sf3\\sf3_output_max.tif;\
                                    dem_sf4\\sf4_output_max.tif",
                                   DATA_PATH, 
                                   "lower_lidar_data.tif", None, "32_BIT_SIGNED", 1, 1, "BLEND", "FIRST")




<Result 'C:\\Users\\mike\\Documents\\OSU\\GEOG581\\Project\\sf_terrain\\MyProject1\\test_raster_1.tif'>

In [32]:
sf_lower_raster = arcpy.Raster("lower_lidar_data.tif")
sf_output_max_corr = sf_lower_raster + 32.6
sf_output_max_corr.save("lower_lidar_data_corrected.tif")

In [33]:
arcpy.management.MosaicToNewRaster("dem_sf1\\sf1_output_upper.tif;\
                                    dem_sf0\\sf0_output_max.tif;\
                                    lower_lidar_data_corrected.tif",
                                   DATA_PATH, 
                                   "lidar_data_corrected.tif", None, "32_BIT_SIGNED", 1, 1, "BLEND", "FIRST")

<Result 'C:\\Users\\mike\\Documents\\OSU\\GEOG581\\Project\\sf_terrain\\MyProject1\\lidar_data_corrected.tif'>

In [34]:
arcpy.ddd.Minus("lidar_data_corrected.tif", "sf_ned", "ground_height.tif")

<Result 'C:\\Users\\mike\\Documents\\OSU\\GEOG581\\Project\\sf_terrain\\MyProject1\\ground_height.tif'>